## Visualization of Robot missions

In [ ]:
from pathlib import Path
import pandas as pd
import contextily as cx
import folium
import branca.colormap as cm
import matplotlib.pyplot as plt
import pickle
from matplotlib.ticker import FormatStrFormatter
import matplotlib.gridspec as gridspec

In [ ]:
input_data_folder_base = Path("../results/")
input_data_folder = Path.cwd() / input_data_folder_base
overall_collection_file_name = "overall_collection_2023.p"
key_parameter_file_name = "key_parameter_2023.p"
cluster_positions_file_name = "cluster_positions_2023.p"
print(f"Reading raw pandas files from \n    {input_data_folder}")

output_folder_base = Path("../html/")
output_image_folder = output_folder_base / Path("./images/")
print(f"Writing website information to \n    {output_folder_base}")

In [ ]:
df_results = pd.read_pickle(input_data_folder / overall_collection_file_name)
df_results.head(3)

In [ ]:
cluster_position_file = input_data_folder / Path(cluster_positions_file_name)
with open(cluster_position_file, 'rb') as fp:
    centroids = pickle.load(fp)
centroids


In [21]:
map = folium.Map(location = [df_results.lat_est.mean(),  
                            df_results.lon_est.mean()], 
                            tiles='Stamen Terrain', zoom_start = 8)

basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

basemaps['Google Satellite Hybrid'].add_to(map)

for key in centroids:

    map.add_child(folium.CircleMarker(location = [centroids[key][0][0],
                                                  centroids[key][0][1]],
                                    color = 'red',
                                    tooltip= f"Click to zoom to {key}",
                                    popup=f"<a href=\"https://sebastianzug.github.io/RoBiMo_Trop_DataSet/html/{key.lower()}.html\" onclick=\"window.open(url, '_blank').focus();\">{key}</a>",
                                    radius = 10, fill=True))

filename = "index.html"
html_folder = Path("../html/")
html_file = Path.cwd() / html_folder / Path(filename)
map.save(html_file)

map
